In [1]:
#kütüphaneleri import edelim
import cv2
import numpy as np
from collections import deque

# algoritması
1-) paint yardımıyla mavi renginin max ve min degerlerini buldum 
2-) webcam baglandim
3-) webcam goruntumu hsv formatina cevirdim
4-) mask uygulayarak mavi renkteni nesnemi tespit ettim daha sonra onu netleştirmek için erezyon uyguladım
5-) hierarchy ile en dıştaki konturununu bulup nesnemi dikdortgen icerisine aldim
6-) merkezini buularak daha önce merkezler neredeyse onu ekrana yazdıracak sekilde bir for dongusu olusturdum


In [3]:
buffer_size= 16
pts = deque(maxlen = buffer_size)

# mavi renk aralığı belirleyecegim
blueLower = (84, 98, 0)
blueUpper= (179, 255, 255)

#webcam baglan
cap = cv2.VideoCapture(0)
cap.set(3,960)
cap.set(4,480)


while True:
    success, imgOriginal= cap.read()
    
    #eger fralemeri okuyorsam islem yap
    if success:
        #flameleri blurla 
        bulurred= cv2.GaussianBlur(imgOriginal, (11,11),0)
        
        #blurlanan resmi hsv cevir
        hsv=cv2.cvtColor(bulurred, cv2.COLOR_BGR2HSV)
        cv2.imshow("hsv image", hsv)
        
        #mavi icin mask olusturuyorum
        mask=cv2.inRange(hsv,blueLower,blueUpper)
        cv2.imshow("mask image", mask)
        
        #maskenin etrafında kalan erezyonlari sil
        mask = cv2.erode(mask, None, iterations= 2 )
        mask= cv2.dilate(mask, None, iterations=2)
        cv2.imshow("mask + erezyon ", mask )
        
        #konturlari bulucam
        contours, _ = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        center = None
        
        #eger konturlari bulduysam
        if len(contours)>0:
            
            #en buyuk konturu al
            c=max(contours, key = cv2.contourArea)
            
            #dikdortgene cevir
            rect = cv2.minAreaRect(c)
            
            #rectengle bir tupledir içerisinde (x-y) kordinatları, (yukseklik- genislik) ve rotasyon vardir
            ((x,y),(width,height),rotation) =  rect
            
            s= "x: {}, y:{}, width:{}, height:{}, rotation:{}".format(np.round(x),np.round(y),np.round(width),np.round(height),np.round(rotation))
            print(s)
            
            #kutu
            box = cv2.boxPoints(rect)
            box = np.int64(box)
            
            #moment: goruntu momenti ile yarıçap, agirlik merkezi gibi ozellikler bulunur
            M = cv2.moments(c)
            center=(int(M["m10"]/M["m00"]),int(M["m01"]/M["m00"]))#nesnemin merkezini buldum
            
            #kontur cizdir
            cv2.drawContours(imgOriginal, [box], 0, (0,255,255),2)
            
            #merkeze bir tane nokta ciz     (yaricap, renk, ici dolu)
            cv2.circle((0,255,255), center, 5, (0,255,255), -1)
            
            #bilgileri ekrana yazdir(yazilacak yer, yazılan bilgi, kordinat, font,buyukluk, renk, kalinlik )
            cv2.putText(imgOriginal,s,(50,50), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1,(0,0,0),2 )
            
        #takip algoritması yapacagiz 
        pts.appendleft(center)
        
        for i in range(1,len(pts)):
            if pts[i-1] is None or pts[i] is None: continue
            cv2.line(imgOriginal, pts[i-1], pts[i],(0,255,0), 3 )
        
        cv2.imshow("image orijial",imgOriginal )
            
            
    #sekmeleri kapat    
    if cv2.waitKey(1) &0xFF == ord("q"): break
           
cap.release()
cv2.destroyAllWindows()
        

x: 250.0, y:290.0, width:4.0, height:4.0, rotation:90.0
x: 259.0, y:293.0, width:4.0, height:4.0, rotation:90.0
x: 265.0, y:291.0, width:4.0, height:4.0, rotation:90.0
x: 286.0, y:286.0, width:6.0, height:7.0, rotation:90.0
x: 301.0, y:282.0, width:5.0, height:4.0, rotation:90.0
x: 303.0, y:282.0, width:4.0, height:4.0, rotation:90.0
x: 310.0, y:268.0, width:5.0, height:7.0, rotation:90.0
x: 229.0, y:257.0, width:4.0, height:4.0, rotation:90.0
x: 273.0, y:272.0, width:4.0, height:4.0, rotation:90.0
x: 79.0, y:406.0, width:52.0, height:11.0, rotation:52.0
x: 99.0, y:375.0, width:55.0, height:11.0, rotation:45.0
x: 120.0, y:346.0, width:60.0, height:11.0, rotation:34.0
x: 115.0, y:311.0, width:121.0, height:12.0, rotation:29.0
x: 131.0, y:298.0, width:129.0, height:12.0, rotation:27.0
x: 148.0, y:286.0, width:132.0, height:11.0, rotation:25.0
x: 163.0, y:278.0, width:132.0, height:11.0, rotation:23.0
x: 175.0, y:271.0, width:133.0, height:11.0, rotation:20.0
x: 189.0, y:263.0, width:137.

x: 62.0, y:201.0, width:9.0, height:124.0, rotation:86.0
x: 55.0, y:189.0, width:11.0, height:111.0, rotation:84.0
x: 50.0, y:182.0, width:12.0, height:102.0, rotation:84.0
x: 47.0, y:178.0, width:12.0, height:95.0, rotation:83.0
x: 45.0, y:175.0, width:13.0, height:91.0, rotation:83.0
x: 44.0, y:175.0, width:13.0, height:90.0, rotation:83.0
x: 44.0, y:174.0, width:13.0, height:89.0, rotation:83.0
x: 44.0, y:175.0, width:13.0, height:90.0, rotation:84.0
x: 45.0, y:175.0, width:13.0, height:92.0, rotation:84.0
x: 46.0, y:175.0, width:13.0, height:94.0, rotation:84.0
x: 48.0, y:175.0, width:13.0, height:97.0, rotation:83.0
x: 49.0, y:176.0, width:13.0, height:100.0, rotation:84.0
x: 51.0, y:177.0, width:13.0, height:104.0, rotation:84.0
x: 53.0, y:178.0, width:13.0, height:107.0, rotation:84.0
x: 54.0, y:177.0, width:13.0, height:110.0, rotation:84.0
x: 56.0, y:177.0, width:13.0, height:113.0, rotation:84.0
x: 58.0, y:176.0, width:13.0, height:117.0, rotation:84.0
x: 60.0, y:174.0, width

x: 349.0, y:166.0, width:13.0, height:200.0, rotation:89.0
x: 346.0, y:167.0, width:14.0, height:201.0, rotation:89.0
x: 344.0, y:172.0, width:13.0, height:202.0, rotation:89.0
x: 343.0, y:184.0, width:206.0, height:15.0, rotation:1.0
x: 343.0, y:191.0, width:206.0, height:15.0, rotation:1.0
x: 342.0, y:200.0, width:208.0, height:15.0, rotation:2.0
x: 342.0, y:200.0, width:208.0, height:15.0, rotation:2.0
x: 340.0, y:211.0, width:208.0, height:14.0, rotation:2.0
x: 338.0, y:223.0, width:210.0, height:13.0, rotation:1.0
x: 334.0, y:236.0, width:212.0, height:13.0, rotation:2.0
x: 331.0, y:250.0, width:212.0, height:13.0, rotation:3.0
x: 327.0, y:267.0, width:214.0, height:14.0, rotation:3.0
x: 323.0, y:284.0, width:216.0, height:15.0, rotation:4.0
x: 341.0, y:304.0, width:174.0, height:13.0, rotation:4.0
x: 391.0, y:333.0, width:67.0, height:12.0, rotation:5.0
x: 382.0, y:382.0, width:67.0, height:13.0, rotation:6.0
x: 372.0, y:401.0, width:71.0, height:14.0, rotation:7.0
x: 364.0, y:41

x: 102.0, y:246.0, width:205.0, height:13.0, rotation:2.0
x: 100.0, y:268.0, width:201.0, height:12.0, rotation:4.0
x: 98.0, y:296.0, width:198.0, height:18.0, rotation:6.0
x: 90.0, y:343.0, width:188.0, height:20.0, rotation:11.0
x: 86.0, y:353.0, width:179.0, height:19.0, rotation:14.0
x: 82.0, y:353.0, width:172.0, height:19.0, rotation:14.0
x: 82.0, y:342.0, width:169.0, height:19.0, rotation:13.0
x: 85.0, y:332.0, width:175.0, height:19.0, rotation:9.0
x: 93.0, y:328.0, width:192.0, height:19.0, rotation:10.0
x: 102.0, y:331.0, width:212.0, height:19.0, rotation:11.0
x: 108.0, y:345.0, width:224.0, height:19.0, rotation:16.0
x: 108.0, y:364.0, width:223.0, height:18.0, rotation:22.0
x: 95.0, y:384.0, width:216.0, height:19.0, rotation:26.0
x: 85.0, y:387.0, width:199.0, height:20.0, rotation:27.0
x: 80.0, y:372.0, width:181.0, height:21.0, rotation:28.0
x: 77.0, y:347.0, width:168.0, height:19.0, rotation:19.0
x: 80.0, y:316.0, width:168.0, height:18.0, rotation:16.0
x: 86.0, y:29

x: 335.0, y:298.0, width:220.0, height:15.0, rotation:10.0
x: 330.0, y:284.0, width:217.0, height:15.0, rotation:8.0
x: 323.0, y:266.0, width:216.0, height:14.0, rotation:8.0
x: 315.0, y:246.0, width:217.0, height:12.0, rotation:5.0
x: 308.0, y:226.0, width:218.0, height:12.0, rotation:4.0
x: 303.0, y:214.0, width:219.0, height:14.0, rotation:2.0
x: 304.0, y:214.0, width:219.0, height:14.0, rotation:2.0
x: 370.0, y:234.0, width:102.0, height:12.0, rotation:2.0
x: 334.0, y:263.0, width:201.0, height:12.0, rotation:5.0
x: 302.0, y:288.0, width:138.0, height:15.0, rotation:8.0
x: 349.0, y:323.0, width:216.0, height:14.0, rotation:12.0
x: 352.0, y:335.0, width:217.0, height:14.0, rotation:12.0
x: 351.0, y:332.0, width:216.0, height:14.0, rotation:12.0
x: 349.0, y:319.0, width:215.0, height:14.0, rotation:10.0
x: 307.0, y:292.0, width:136.0, height:14.0, rotation:7.0
x: 338.0, y:271.0, width:210.0, height:13.0, rotation:6.0
x: 371.0, y:252.0, width:134.0, height:12.0, rotation:5.0
x: 325.0,

...

x: 245.0, y:243.0, width:14.0, height:12.0, rotation:90.0
x: 264.0, y:248.0, width:14.0, height:13.0, rotation:90.0
x: 278.0, y:254.0, width:18.0, height:13.0, rotation:90.0
x: 285.0, y:254.0, width:17.0, height:14.0, rotation:90.0
x: 295.0, y:254.0, width:18.0, height:16.0, rotation:90.0
x: 302.0, y:254.0, width:19.0, height:17.0, rotation:90.0
x: 307.0, y:253.0, width:24.0, height:14.0, rotation:45.0
x: 312.0, y:248.0, width:28.0, height:13.0, rotation:45.0
x: 316.0, y:248.0, width:24.0, height:13.0, rotation:45.0
x: 315.0, y:245.0, width:24.0, height:13.0, rotation:45.0
x: 313.0, y:244.0, width:15.0, height:18.0, rotation:90.0
x: 312.0, y:241.0, width:14.0, height:17.0, rotation:90.0
x: 310.0, y:238.0, width:14.0, height:17.0, rotation:90.0
x: 306.0, y:232.0, width:13.0, height:14.0, rotation:90.0
x: 302.0, y:224.0, width:13.0, height:13.0, rotation:90.0
x: 298.0, y:217.0, width:12.0, height:13.0, rotation:90.0
x: 294.0, y:211.0, width:12.0, height:13.0, rotation:90.0
x: 293.0, y:20

x: 153.0, y:237.0, width:117.0, height:11.0, rotation:11.0
x: 197.0, y:257.0, width:202.0, height:15.0, rotation:14.0
x: 154.0, y:246.0, width:117.0, height:11.0, rotation:15.0
x: 142.0, y:233.0, width:103.0, height:10.0, rotation:11.0
x: 120.0, y:215.0, width:77.0, height:8.0, rotation:6.0
x: 225.0, y:192.0, width:19.0, height:60.0, rotation:90.0
x: 151.0, y:171.0, width:14.0, height:191.0, rotation:81.0
x: 152.0, y:174.0, width:16.0, height:191.0, rotation:82.0
x: 221.0, y:204.0, width:21.0, height:68.0, rotation:90.0
x: 167.0, y:230.0, width:195.0, height:13.0, rotation:11.0
x: 135.0, y:224.0, width:131.0, height:10.0, rotation:11.0
x: 117.0, y:205.0, width:109.0, height:9.0, rotation:5.0
x: 153.0, y:188.0, width:15.0, height:191.0, rotation:89.0
x: 155.0, y:182.0, width:14.0, height:192.0, rotation:85.0
x: 162.0, y:198.0, width:17.0, height:190.0, rotation:90.0
x: 168.0, y:216.0, width:193.0, height:13.0, rotation:7.0
x: 167.0, y:216.0, width:193.0, height:12.0, rotation:6.0
x: 161

x: 174.0, y:273.0, width:12.0, height:12.0, rotation:90.0
x: 181.0, y:274.0, width:13.0, height:12.0, rotation:90.0
x: 189.0, y:275.0, width:14.0, height:14.0, rotation:90.0
x: 188.0, y:272.0, width:13.0, height:13.0, rotation:90.0
x: 192.0, y:272.0, width:13.0, height:14.0, rotation:90.0
x: 197.0, y:274.0, width:13.0, height:14.0, rotation:90.0
x: 195.0, y:273.0, width:14.0, height:14.0, rotation:90.0
x: 192.0, y:274.0, width:13.0, height:15.0, rotation:90.0
x: 192.0, y:276.0, width:14.0, height:13.0, rotation:90.0
x: 190.0, y:276.0, width:13.0, height:13.0, rotation:90.0
x: 188.0, y:278.0, width:14.0, height:13.0, rotation:90.0
x: 186.0, y:278.0, width:13.0, height:13.0, rotation:90.0
x: 182.0, y:276.0, width:13.0, height:13.0, rotation:90.0
x: 175.0, y:276.0, width:13.0, height:12.0, rotation:90.0
x: 170.0, y:274.0, width:12.0, height:11.0, rotation:90.0
x: 164.0, y:272.0, width:11.0, height:11.0, rotation:90.0
x: 163.0, y:274.0, width:11.0, height:10.0, rotation:90.0
x: 160.0, y:27

x: 143.0, y:349.0, width:12.0, height:31.0, rotation:16.0
x: 124.0, y:348.0, width:12.0, height:37.0, rotation:18.0
x: 107.0, y:350.0, width:13.0, height:43.0, rotation:29.0
x: 90.0, y:346.0, width:14.0, height:43.0, rotation:27.0
x: 82.0, y:335.0, width:14.0, height:42.0, rotation:34.0
x: 78.0, y:318.0, width:13.0, height:38.0, rotation:45.0
x: 74.0, y:299.0, width:13.0, height:34.0, rotation:45.0
x: 75.0, y:281.0, width:13.0, height:26.0, rotation:48.0
x: 82.0, y:266.0, width:13.0, height:20.0, rotation:90.0
x: 82.0, y:266.0, width:13.0, height:20.0, rotation:90.0
x: 88.0, y:250.0, width:12.0, height:16.0, rotation:90.0
x: 114.0, y:239.0, width:10.0, height:14.0, rotation:90.0
x: 134.0, y:246.0, width:9.0, height:10.0, rotation:90.0
x: 136.0, y:248.0, width:11.0, height:11.0, rotation:90.0
x: 136.0, y:254.0, width:10.0, height:10.0, rotation:90.0
x: 130.0, y:270.0, width:13.0, height:11.0, rotation:90.0
x: 114.0, y:290.0, width:17.0, height:15.0, rotation:90.0
x: 106.0, y:298.0, widt

x: 26.0, y:472.0, width:4.0, height:4.0, rotation:90.0
x: 284.0, y:292.0, width:4.0, height:4.0, rotation:90.0
x: 286.0, y:292.0, width:4.0, height:4.0, rotation:90.0
